In [ ]:
# -*- coding: utf-8
# Repórter Brasil (https://ruralometro2022.reporterbrasil.org.br/)
# Reinaldo Chaves (@paidatocandeira)
# # Procura empresas de deputados federais inscritas em embargos do Ibama

In [1]:
import pandas as pd
import unidecode

In [2]:
def f(str):
    try:
        return (unidecode.unidecode(str))
    except:
        return (str)

In [3]:
ibama = pd.read_excel('originais/embargos_ibama_1_fev_2022.xlsx', sheet_name='Planilha1')
ibama = ibama[pd.notnull(ibama['CPF ou CNPJ'])]
ibama.rename(columns = {'CPF ou CNPJ':'cpf_cnpj'},inplace = True)
def limpeza1(linha):
    documento = str(linha["cpf_cnpj"])
    documento = documento.strip()
    documento = documento.replace('.', '')
    documento = documento.replace('-', '')
    documento = documento.replace('/', '')
    return str(documento)
ibama["cpf_cnpj"] = ibama.apply(limpeza1, axis=1)
# Preenche zeros a esquerda nos documentos
def compara1(linha):
    documento = str(linha["cpf_cnpj"])
    
    if len(documento) < 14 and len(documento) >= 12:
        conta = 14 - (len(documento))
        zero = '0'
        zeros = zero * conta
        alterado = zeros + documento
    elif len(documento) == 14:
        alterado = documento
    elif len(documento) < 11:
        conta = 11 - (len(documento))
        zero = '0'
        zeros = zero * conta
        alterado = zeros + documento
    elif len(documento) == 11:
        alterado = documento
    
    return alterado
ibama["cpf_cnpj"] = ibama.apply(compara1, axis=1)
def limpeza2(linha):
    documento = str(linha["cpf_cnpj"])
    if len(documento) > 11:
        return documento[:8]
    else:
        return "documento_cpf"

ibama['cnpj_raiz'] = ibama.apply(limpeza2, axis=1)
ibama_pf = ibama[(ibama['cnpj_raiz'] == 'documento_cpf')].copy()
ibama_pj = ibama[(ibama['cnpj_raiz'] != 'documento_cpf')].copy()

In [4]:
kwargs = {'sep': ',', 'dtype': str, 'encoding': 'utf-8'}
empresas_politicos = pd.read_csv("resultados/empresas_deputados/empresas_deputados_fev_2022.csv", **kwargs)

In [5]:
kwargs = {'sep': ',', 'dtype': str, 'encoding': 'utf-8'}
empresas_politicos_socios = pd.read_csv("resultados/empresas_deputados/todos_socios_empresas_deputados_fev_2022.csv", **kwargs)

In [20]:
# Procura empresas de deputados em embargos do Ibama

In [6]:
empresas_politicos.columns

Index(['nome', 'nome_urna', 'politico_id', 'cpf_mascarado', 'cnpj',
       'cnpj_basico_x', 'nome_socio', 'cnpj_cpf_socio', 'razao_social',
       'nome_fantasia', 'situacao_cadastral', 'cnae_fiscal',
       'cnae_fiscal_secundaria', 'uf', 'municipio'],
      dtype='object')

In [7]:
empresas_ibama = pd.merge(empresas_politicos.drop_duplicates('cnpj_basico_x'), ibama_pj.drop_duplicates('cnpj_raiz'), left_on='cnpj_basico_x', right_on='cnpj_raiz', how = 'left')

In [8]:
conta = empresas_ibama[pd.notnull(empresas_ibama['cpf_cnpj'])].copy()
conta.shape

(2, 31)

In [24]:
conta

,nome,nome_urna,politico_id,cpf_mascarado,cnpj,cnpj_basico_x,nome_socio,cnpj_cpf_socio,nome_fantasia,situacao_cadastral,...,Localização do Imóvel,UF Infração,Município Infração,Município Infrator,Bairro,Endereço,Julgamento,Infração,Data de Inserção na Lista,cnpj_raiz
142,GUTEMBERG REIS DE OLIVEIRA,Gutemberg Reis,07733341736,***333417**,02880684000199,02880684,GUTEMBERG REIS DE OLIVEIRA,***333417**,MEGA EMPREENDIMENTOS,02,...,"Rua Eneas Frutuoso , 33 , Xerem / Duque de Cax...",RJ,Duque de Caxias,Duque de Caxias/RJ,Xerém,"Est. Beira Rio, S/N,",pendente de julgamento,"Implantar projetos de loteamentos, sem a licen...",2008-03-26,02880684
257,MAGDA MOFATTO HON,Magda Mofatto,13205285115,***052851**,03465317000191,03465317,MAGDA MOFATTO HON,***052851**,NaN,02,...,Di Roma Hotel Clube.,GO,Caldas Novas,Caldas Novas/GO,Solar de Caldas,Rua Sao Cristovao,2012-01-13 00:00:00,"Destruir (danificar, desmatar) florestas ou de...",2005-11-23,03465317


In [25]:
conta.to_excel('resultados/empresas_embargos/empresas de deputados em embargos do Ibama.xlsx',sheet_name='Sheet1',index=False)

In [26]:
conta.columns

Index(['nome', 'nome_urna', 'politico_id', 'cpf_mascarado', 'cnpj',
       'cnpj_basico_x', 'nome_socio', 'cnpj_cpf_socio', 'nome_fantasia',
       'situacao_cadastral', 'cnae_fiscal', 'cnae_fiscal_secundaria', 'uf',
       'municipio', 'Nº TAD', 'SERIE TAD', 'Área (ha)', 'Nº A.I.',
       'Nome ou Razão Social', 'cpf_cnpj', 'Localização do Imóvel',
       'UF Infração', 'Município Infração', 'Município Infrator', 'Bairro',
       'Endereço', 'Julgamento', 'Infração', 'Data de Inserção na Lista',
       'cnpj_raiz'],
      dtype='object')

In [27]:
conta.rename(columns = {'cnpj':'entidade_id'},inplace = True)
conta.rename(columns = {'nome':'politico_nome'},inplace = True)
conta.rename(columns = {'Nome ou Razão Social':'entidade_nome'},inplace = True)

In [28]:
conta = conta[['politico_id', 'entidade_id', 'politico_nome', 'entidade_nome']]

In [29]:
conta.to_csv('resultados/empresas_embargos/empresas de deputados em embargos do Ibama.csv',index=False)

In [26]:
# Procura sócio de empresas de deputados em embargos do Ibama

In [30]:
empresas_politicos_socios.columns

Index(['cnpj', 'cnpj_basico_x', 'cnpj_basico', 'identificador_de_socio',
       'nome_socio', 'cnpj_cpf_socio', 'qualificacao_socio', 'pais',
       'representante_legal', 'nome_representante'],
      dtype='object')

In [31]:
empresas_politicos_socios['cnpj_cpf_socio']

0          ***564768**
1          ***921272**
2          ***001738**
3          ***381948**
4          ***916458**
             ...      
2096       ***148156**
2097       ***858106**
2098    17845504000105
2099       ***167658**
2100       ***070616**
Name: cnpj_cpf_socio, Length: 2101, dtype: object

In [32]:
ibama.columns

Index(['Nº TAD', 'SERIE TAD', 'Área (ha)', 'Nº A.I.', 'Nome ou Razão Social',
       'cpf_cnpj', 'Localização do Imóvel', 'UF Infração',
       'Município Infração', 'Município Infrator', 'Bairro', 'Endereço',
       'Julgamento', 'Infração', 'Data de Inserção na Lista', 'cnpj_raiz'],
      dtype='object')

In [33]:
ibama_pf['cpf_mascarado_cpf_cnpj']= '***'+ibama_pf['cpf_cnpj'].str[3:9]+'**'
ibama_pf['nome_semacentos'] = ibama_pf['Nome ou Razão Social'].str.upper()
ibama_pf['nome_semacentos'] = ibama_pf['nome_semacentos'].apply(f)

In [34]:
ibama_pf['cpf_mascarado_cpf_cnpj']

13       ***639369**
14       ***106922**
15       ***098182**
16       ***356713**
17       ***856276**
            ...     
76644    ***490366**
76647    ***522127**
76649    ***416291**
76661    ***378442**
76671    ***025358**
Name: cpf_mascarado_cpf_cnpj, Length: 57304, dtype: object

In [35]:
socios_ibama = pd.merge(empresas_politicos_socios.drop_duplicates('cnpj_cpf_socio'), ibama_pf.drop_duplicates('cpf_cnpj'), left_on=['cnpj_cpf_socio', 'nome_socio'], right_on=['cpf_mascarado_cpf_cnpj','nome_semacentos'], how = 'left')

In [41]:
conta = socios_ibama[pd.notnull(socios_ibama['cpf_cnpj'])].copy()
conta.shape

(13, 28)

In [42]:
conta

,cnpj,cnpj_basico_x,cnpj_basico,identificador_de_socio,nome_socio,cnpj_cpf_socio,qualificacao_socio,pais,representante_legal,nome_representante,...,Município Infração,Município Infrator,Bairro,Endereço,Julgamento,Infração,Data de Inserção na Lista,cnpj_raiz,cpf_mascarado_cpf_cnpj,nome_semacentos
35,16733548000180,16733548,16733548,2,NILTON ALCANTARA NEVES,***783131**,49,NaN,***000000**,NaN,...,São Félix do Xingu,Palmas/TO,Plano Diretor Sul,Qd 704 Sul Av Ns 04 Lt 19-A Pac,pendente de julgamento,"Destruir, desmatar, danificar florestas ou qua...",2010-12-15,documento_cpf,***783131**,NILTON ALCANTARA NEVES
62,29300502000110,29300502,29300502,2,IVO NARCISO CASSOL,***766409**,22,NaN,***000000**,NaN,...,Alta Floresta D'Oeste,Rolim de Moura/RO,Centro,Av Macapa,2014-05-20 00:00:00,"Destruir (danificar, desmatar) florestas ou de...",2007-04-26,documento_cpf,***766409**,IVO NARCISO CASSOL
104,00298270000120,00298270,00298270,2,LUCIANO CALDAS BIVAR,***189614**,5,NaN,***000000**,NaN,...,Jaboatão dos Guararapes,Jaboatão dos Guararapes/PE,Candeias,"Av.Bernardo de Vieira de Melo, 1626, Aptº 1301",2016-09-08 00:00:00,"Promover construção em solo não edificável, ou...",2012-08-08,documento_cpf,***189614**,LUCIANO CALDAS BIVAR
160,06167010000100,06167010,06167010,2,JOAO BATISTA DIAS PEREIRA,***540891**,22,NaN,***000000**,NaN,...,Palmas,Palmas/TO,Plano Diretor Sul,"Qd 404 Sul, Al.02, Apto 1702",pendente de julgamento,NaN,2007-07-26,documento_cpf,***540891**,JOAO BATISTA DIAS PEREIRA
245,24829688000185,24829688,24829688,2,CESAR FARID FIAT,***346807**,5,NaN,***000000**,NaN,...,Silva Jardim,Araruama/RJ,Pontinha,Rua Otávio Carneiro,2018-06-25 00:00:00,Usar fogo em qualquer forma de vegetação (exce...,2009-02-19,documento_cpf,***346807**,CESAR FARID FIAT
276,05143490000107,05143490,05143490,2,HELDER ZAHLUTH BARBALHO,***943702**,22,NaN,***000000**,NaN,...,Ananindeua,Ananindeua/PA,Centro,"Av. Magalhães Barata, 115",pendente de julgamento,Causar poluição de qualquer natureza em níveis...,2013-01-08,documento_cpf,***943702**,HELDER ZAHLUTH BARBALHO
315,13836713000131,13836713,13836713,2,JOSE MILTON DE ABREU,***546255**,22,NaN,***000000**,NaN,...,Euclides da Cunha,Euclides da Cunha/BA,Centro,Praça da Bandeira Nº 102,2013-08-29 00:00:00,"Destruir, desmatar, danificar florestas ou qua...",2011-01-04,documento_cpf,***546255**,JOSE MILTON DE ABREU
398,00044180000103,00044180,00044180,2,LUIZ GONZAGA PATRIOTA,***609704**,16,NaN,***000000**,NaN,...,Petrolina,Brasília/DF,Asa Norte,"Sqn 302 - Bloco 'F', Aptº 402",2018-09-13 00:00:00,Infração da Flora(Não Classificada-Móvel),2017-04-04,documento_cpf,***609704**,LUIZ GONZAGA PATRIOTA
417,83307348000153,83307348,83307348,2,ERNANI MALDANER,***953629**,49,NaN,***000000**,NaN,...,Santarém,Aveiro/PA,Zona Rural,Lote de Terra Rural Nº 08 Setor e,2015-05-11 00:00:00,Infração da Flora(Não Classificada-Móvel),2015-04-02,documento_cpf,***953629**,ERNANI MALDANER
433,03220504000105,03220504,03220504,2,JOSE ALVES ROCHA,***891135**,49,NaN,***000000**,NaN,...,Coribe,Salvador/BA,Vitoria,Av. Sete de Setembro Nº 1796 Apt. 501,pendente de julgamento,Desmatar florestas ou demais formas de vegetaç...,2008-03-14,documento_cpf,***891135**,JOSE ALVES ROCHA


In [43]:
conta.to_excel('resultados/empresas_embargos/Sócios de empresas de deputados em embargos do Ibama.xlsx',sheet_name='Sheet1',index=False)

In [44]:
conta.columns

Index(['cnpj', 'cnpj_basico_x', 'cnpj_basico', 'identificador_de_socio',
       'nome_socio', 'cnpj_cpf_socio', 'qualificacao_socio', 'pais',
       'representante_legal', 'nome_representante', 'Nº TAD', 'SERIE TAD',
       'Área (ha)', 'Nº A.I.', 'Nome ou Razão Social', 'cpf_cnpj',
       'Localização do Imóvel', 'UF Infração', 'Município Infração',
       'Município Infrator', 'Bairro', 'Endereço', 'Julgamento', 'Infração',
       'Data de Inserção na Lista', 'cnpj_raiz', 'cpf_mascarado_cpf_cnpj',
       'nome_semacentos'],
      dtype='object')

In [45]:
conta.rename(columns = {'cnpj_cpf_socio':'entidade_id'},inplace = True)
conta.rename(columns = {'nome_socio':'entidade_nome'},inplace = True)

In [46]:
conta = conta[['entidade_id', 'entidade_nome']]

In [47]:
conta.to_csv('resultados/empresas_embargos/sócios de empresas de deputados em embargos do Ibama.csv',index=False)